In [6]:
from otp_client import OTPClient
import pandas as pd
from datetime import datetime

In [7]:
# bike_id                                        613703
# from_station                    Emilii Plater - Złota
# from_station_lat                            52.230978
# from_station_lng                            21.003676
# to_station          Marszałkowska - Al. Jerozolimskie
# to_station_lat                              52.229286
# to_station_lng                              21.011579
# departure_time                    2025-04-02 19:44:13
# arrival_time                      2025-04-02 19:45:14

In [8]:
def print_statisticks(itinerary):    
    mins, secs = divmod(itinerary["duration"], 60)
    print(
        f"Fastest trip takes {int(mins)} min {int(secs)} s across"
        f" {len(itinerary['legs'])} legs."
    )

    for leg in itinerary["legs"]:
        frm, to = leg["from"], leg["to"]
        print(
            f"{leg['mode']:7s}"
            f" {frm['lat']:.5f},{frm['lon']:.5f} → {to['lat']:.5f},{to['lon']:.5f}"
        )

In [10]:
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

otp = OTPClient()

warsaw_bike_movements = pd.read_csv('..\\bike_movements_warsaw.csv')
trips_that_didnt_go_anywhere = []
trips_that_went_somewhere = []
other_errors = []
for index, row in tqdm(warsaw_bike_movements.iterrows(), total=warsaw_bike_movements.shape[0]):
    start_lat = row['from_station_lat']
    start_lng = row['from_station_lng']
    end_lat = row['to_station_lat']
    end_lng = row['to_station_lng']
    start_time = row['departure_time']
    end_time = row['arrival_time']

    start_name = row['from_station']
    end_name = row['to_station']

    when = (
            datetime.strptime(start_time.strip(), "%Y-%m-%d %H:%M:%S")
        )
    # Change 'when' to the same day of week, but in the current week (today's week)
    today = datetime.now()
    days_ahead = (row['departure_time'] and datetime.strptime(start_time.strip(), "%Y-%m-%d %H:%M:%S")).weekday() - today.weekday()
    if days_ahead < 0:
        days_ahead += 7
    new_date = today + pd.Timedelta(days=days_ahead)
    when = when.replace(year=new_date.year, month=new_date.month, day=new_date.day)
    
    start_time_time=datetime.strptime(start_time.strip(), "%Y-%m-%d %H:%M:%S")
    end_time_time=datetime.strptime(end_time.strip(), "%Y-%m-%d %H:%M:%S")

    # print(f"From   ||| {start_name} ||| to ||| {end_name}  |||  at {when}")
    # print(f"Bike Duration: {end_time_time-start_time_time} seconds")

    if start_lat == end_lat and start_lng == end_lng:
        # print("Bike did not move anywhere")
        trips_that_didnt_go_anywhere.append(row)
        continue
    


    try:
        itinerary = otp.fastest_connection(when, (start_lat,start_lng), (end_lat,end_lng))
        if itinerary is not None:
            # print("Bike moved somewhere")
            trips_that_went_somewhere.append((row, itinerary))
    except Exception as e:
        print(f"Error: {e} Row: {row}")
        other_errors.append((row, e))
        continue  
    # if len(itinerary['legs'])!= 1:
        # print("QWEQWEQWEQW")
        

    # print("")
    # print("")

  0%|          | 10/193009 [00:04<21:17:31,  2.52it/s]

Error: [{'message': 'Exception while fetching data (/planConnection) : distanceMeters', 'locations': [{'line': 9, 'column': 11}], 'path': ['planConnection'], 'extensions': {'classification': 'DataFetchingException'}}] Row: bike_id                                        611879
from_station                    Metro Księcia Janusza
from_station_lat                            52.239541
from_station_lng                            20.942309
to_station          Jana Olbrachta - Antka Rozpylacza
to_station_lat                              52.234223
to_station_lng                              20.934782
departure_time                    2025-04-02 19:45:14
arrival_time                      2025-04-02 19:50:23
Name: 9, dtype: object


  0%|          | 11/193009 [00:05<27:32:40,  1.95it/s]

Error: [{'message': 'Exception while fetching data (/planConnection) : distanceMeters', 'locations': [{'line': 9, 'column': 11}], 'path': ['planConnection'], 'extensions': {'classification': 'DataFetchingException'}}] Row: bike_id                          612629
from_station            Metro Słodowiec
from_station_lat              52.276157
from_station_lng              20.961667
to_station                    Rondo ONZ
to_station_lat                52.232628
to_station_lng                20.997123
departure_time      2025-04-02 19:48:20
arrival_time        2025-04-02 19:50:23
Name: 10, dtype: object


  0%|          | 16/193009 [00:06<22:50:42,  2.35it/s]


KeyboardInterrupt: 

In [ ]:
import pickle

# Serialize trips_that_didnt_go_anywhere as a DataFrame CSV
pd.DataFrame(trips_that_didnt_go_anywhere).to_csv('otp_trips_data_for_bikes\\trips_that_didnt_go_anywhere2.csv', index=False)

# Serialize trips_that_went_somewhere as a pickle (list of tuples: (row, itinerary))
with open('otp_trips_data_for_bikes\\trips_that_went_somewhere2.pkl', 'wb') as f:
    pickle.dump(trips_that_went_somewhere, f)

# Serialize other_errors as a pickle (list of tuples: (row, exception))
with open('otp_trips_data_for_bikes\\other_errors2.pkl', 'wb') as f:
    pickle.dump(other_errors, f)